In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from importlib.util import find_spec
if find_spec("core") is None:
    import sys
    sys.path.append('..')

import tensorflow as tf
import tensorflow_datasets as tfds

tfds.disable_progress_bar()

In [2]:
ds_train, ds_train_info = tfds.load('RetinaDataset', split='train[:80%]', shuffle_files=True, as_supervised=True,with_info=True)
ds_val, ds_val_info = tfds.load('RetinaDataset', split='train[-20%:]', shuffle_files=True, as_supervised=True,with_info=True)
ds_test, ds_test_info = tfds.load('RetinaDataset', split='test', shuffle_files=True, as_supervised=True,with_info=True)

In [3]:
len(ds_train), len(ds_val), len(ds_test)

(66791, 16698, 968)

In [4]:
def preprocess_img(image, label):
    image = tf.image.convert_image_dtype(image, tf.float32) / 255.
    image = tf.image.resize(image, [224, 224])
    return image, label

In [5]:
ds_train = ds_train.map(preprocess_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
#ds_train = ds_train.cache()
#ds_train = ds_train.shuffle(len(ds_train))
ds_train = ds_train.batch(32)
#ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [6]:
ds_val = ds_val.map(preprocess_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
#ds_val = ds_val.cache()
#ds_train = ds_train.shuffle(len(ds_val))
ds_val = ds_val.batch(32)
#ds_val = ds_val.prefetch(tf.data.experimental.AUTOTUNE)

In [7]:
ds_test = ds_test.map(preprocess_img,
                     num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(32)
#ds_test = ds_test.cache()
#ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [8]:
ds = ds_train.take(1)

for image, label in tfds.as_numpy(ds):
  print(image.shape, label.shape)

(32, 224, 224, 3) (32,)


In [9]:
from core.networks import resnet
model = resnet(input_shape = (224, 224, 3), output_shape = (4,))
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(),
    loss="sparse_categorical_crossentropy",
    metrics=['accuracy'],
)

model.fit(
    ds_train,
    epochs=1,
    validation_data=ds_val,
    verbose=1
)

2088/2088 [==============================] - 363s 174ms/step - loss: 0.5031 - accuracy: 0.8172 - val_loss: 2.9424 - val_accuracy: 0.5546


In [10]:
model.evaluate(ds_test)

31/31 [==============================] - 2s 54ms/step - loss: 4.6464 - accuracy: 0.3874


[4.646430969238281, 0.3873966932296753]